In [220]:
import cv2 as cv
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import networkx as nx
from IPython.display import Audio, display
import ipynb.fs.defs.Utils as Utils

In [221]:
def allDone():
    display(Audio(url='https://sound.peal.io/ps/audios/000/001/131/original/kon.wav', autoplay=True))

In [225]:
def expand_graph(adj_matrix):
    replicas_structure = dict()
    adj_matrix_exp = np.copy(adj_matrix)
    
    #more multiedges, we have to expand
    while np.sum(adj_matrix_exp>1)>0:
        
        adj_matrix_multi = np.maximum(adj_matrix_exp-1,0)
        edges_out = np.sum(adj_matrix_multi, axis=1)
        node_max = np.argmax(edges_out)
        replicas = np.max(adj_matrix_multi[node_max,:])
        appended_column = np.copy(adj_matrix_exp[:,node_max])
        appended_column[node_max]=1
        for i in range(replicas):
            adj_matrix_exp = np.c_[adj_matrix_exp,appended_column]

        row=np.copy(adj_matrix_exp[node_max,:])

        appended_row = np.minimum(1,row)
        adj_matrix_exp[node_max,:] = appended_row
        row = np.maximum(0,row-1)
        for i in range(replicas):
            appended_row = np.minimum(1,row)
            adj_matrix_exp = np.r_[adj_matrix_exp, [appended_row]]
            row = np.maximum(0,row-1)
        
        replicas_structure[node_max] = [node_max] +  list(range(adj_matrix_exp.shape[0]-replicas, adj_matrix_exp.shape[0]))
    
    for i in range(adj_matrix.shape[0]):
        if i not in replicas_structure:
            replicas_structure[i] = [i]
    return adj_matrix_exp, replicas_structure

In [226]:
def compute_Z_matrix(n_expanded, replicas, matches_dict):
    Z = np.eye(3*n_expanded, 3*n_expanded)
    for couple in matches_dict:
        i,j = couple
        for k in range(len(replicas[i])):
            if (k>=len(matches_dict[couple])):
                break
            h = matches_dict[couple][k]
            if j in replicas:
                for w in replicas[j]:
                    Z[3*w:3*(w+1), 3*replicas[i][k]:3*(replicas[i][k]+1)] = h
            else:
                Z[3*j:3*(j+1), 3*replicas[i][k]:3*(replicas[i][k]+1)] = h
                
    for replica in replicas:
        r = replicas[replica]
        for k in r:
            Z[3*k:3*(k+1), 3*r[0]:3*(r[0]+1)] = np.eye(3)

    return Z

In [227]:
def compute_C_matrix(n_expanded, n, replicas):
    C = np.zeros([3*n_expanded, 3*(n_expanded-n)],dtype=int)
    for replica in replicas:
        r = replicas[replica]
        first = r[0]
        for k in r:
            if k != first:
                C[3*first:3*(first+1),3*(k-n):3*(k-n+1)] = np.eye(3)
                C[3*k:3*(k+1),3*(k-n):3*(k-n+1)] = -np.eye(3)

    return C

In [228]:
def compute_M_matrix(adj_matrix, Z):
    n = adj_matrix.shape[0]
    D = np.diag(np.sum(adj_matrix + np.eye(n), axis=1))
    D_kron = np.kron(D, np.eye(3))
    M = Z - D_kron
    return M

In [229]:
def build_multi_graph_matrices(dataset_name,
                imgs,
                matches_dict,
                output_dir ="output",
                verbose = True,
                save_output = True
               ):

    M_filename = f"M_{dataset_name}.npy"
    C_filename = f"C_{dataset_name}.npy"
    Z_filename = f"Z_{dataset_name}.npy"
    Adj_filename = f"Adj_{dataset_name}.npy"
   

    graph_dir = os.path.join(output_dir,'graph')

    if save_output:
        if not os.path.isdir(graph_dir):
            os.makedirs(graph_dir)

    n = len(imgs)


    #compute the adj and weight matrix of matches
    adj_matrix = np.zeros([n,n], dtype=int)
    for (i,j) in matches_dict:
        adj_matrix[i, j] = len(matches_dict[i,j])

    adj_matrix_exp, replicas = expand_graph(adj_matrix)

    if verbose:
        Utils.build_and_print_multi_graph(adj_matrix_exp, save_output, graph_dir, "graph")   #expanded

    n_expanded = adj_matrix_exp.shape[0]
    Z = compute_Z_matrix(n_expanded, replicas, matches_dict)

    C = compute_C_matrix(n_expanded, n, replicas)

    M = compute_M_matrix(adj_matrix_exp, Z)

    if save_output:
        np.save(os.path.join(output_dir,M_filename), M)
        np.save(os.path.join(output_dir,Z_filename), Z)
        np.save(os.path.join(output_dir,C_filename), C)
        np.save(os.path.join(output_dir,Adj_filename), adj_matrix_exp)


    if verbose:#pay attention
        allDone()

    return M, Z, C, adj_matrix_exp
        

In [ ]:
def build_simple_graph_matrices(dataset_name,
                imgs,
                matches_dict,
                output_dir ="output",
                verbose = True,
                save_output = True
               ):
    
    simple_matches_dict = dict()
    for couple in matches_dict:
        simple_matches_dict[couple] = [matches_dict[couple][-1]]
    
    M, Z, _, adj_matrix = build_multi_graph_matrices(dataset_name,
            imgs,
            simple_matches_dict,
            output_dir = output_dir,
            verbose = verbose,
            save_output = save_output
           )

    return M, Z, adj_matrix